# Building Web Applications with Gradio

This tutorial will guide you through using Gradio to build interactive web applications from your Python functions. We'll cover Gradio's design philosophy, main components, and deployment options.

## What is Gradio?

Gradio is an open-source Python library that makes it easy to create customizable UI components for machine learning models, APIs, or any Python function. It's designed to make sharing your work as simple as possible.

### Design Philosophy

Gradio follows several key principles:

1. **Simplicity First**: Convert any Python function into a web interface with just a few lines of code
2. **Zero Web Development Knowledge Required**: No HTML, CSS, or JavaScript needed
3. **Rapid Prototyping**: Get a working demo in minutes, not hours
4. **Flexible Components**: Rich set of input/output components for different data types
5. **Easy Sharing**: Built-in sharing capabilities and deployment options

## Installation and Setup

First, let's install Gradio and import the necessary libraries:

In [2]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
import base64

## Core Concepts

### 1. The Basic Pattern

Every Gradio app follows this pattern:
1. Define a Python function
2. Create an interface with `gr.Interface()`
3. Launch the app with `.launch()`

Let's start with the classic "Hello World" example:

In [3]:
def greet(name):
    return f"Hello, {name}!"

# Create the interface
demo = gr.Interface(
    fn=greet,
    inputs="text",
    outputs="text",
    title="Hello World App",
    description="Enter your name to get a greeting!"
)

# Launch the app
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Created dataset file at: .gradio\flagged\dataset1.csv


### 2. Input and Output Components

Gradio provides a rich set of components for different data types:

In [4]:
def calculator(operation, num1, num2):
    if operation == "Add":
        return num1 + num2
    elif operation == "Subtract":
        return num1 - num2
    elif operation == "Multiply":
        return num1 * num2
    elif operation == "Divide":
        if num2 != 0:
            return num1 / num2
        else:
            return "Error: Division by zero!"

calculator_demo = gr.Interface(
    fn=calculator,
    inputs=[
        gr.Dropdown(["Add", "Subtract", "Multiply", "Divide"], label="Operation"),
        gr.Number(label="First Number"),
        gr.Number(label="Second Number")
    ],
    outputs=gr.Number(label="Result"),
    title="Simple Calculator",
    description="Perform basic arithmetic operations"
)

calculator_demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


## Main Components

### Input Components

- **Text**: `gr.Textbox()` - for text input
- **Number**: `gr.Number()` - for numeric input
- **Slider**: `gr.Slider()` - for selecting values within a range
- **Dropdown**: `gr.Dropdown()` - for selecting from options
- **Checkbox**: `gr.Checkbox()` - for boolean input
- **Radio**: `gr.Radio()` - for selecting one option
- **Image**: `gr.Image()` - for image upload
- **Audio**: `gr.Audio()` - for audio upload
- **File**: `gr.File()` - for file upload

### Output Components

- **Text**: `gr.Textbox()` - for text output
- **Label**: `gr.Label()` - for classification results
- **Image**: `gr.Image()` - for image output
- **Plot**: `gr.Plot()` - for matplotlib plots
- **HTML**: `gr.HTML()` - for HTML content
- **JSON**: `gr.JSON()` - for JSON data
- **Dataframe**: `gr.Dataframe()` - for tabular data

## Practical Examples

### Example 1: Data Analysis Tool

Let's create a data analysis tool using the penguins dataset:

In [ ]:
# Load the penguins dataset
penguins = pd.read_csv('data/penguins.csv')

def analyze_penguins(species, plot_type):
    # Filter data if specific species selected
    if species != "All":
        data = penguins[penguins['species'] == species]
    else:
        data = penguins
    
    # Create plot
    plt.figure(figsize=(10, 6))
    
    if plot_type == "Body Mass Distribution":
        if species == "All":
            sns.histplot(data=data, x='body_mass_g', hue='species', bins=20)
        else:
            sns.histplot(data=data, x='body_mass_g', bins=20)
        plt.title(f'Body Mass Distribution - {species}')
        plt.xlabel('Body Mass (g)')
        
    elif plot_type == "Bill Length vs Depth":
        if species == "All":
            sns.scatterplot(data=data, x='bill_length_mm', y='bill_depth_mm', hue='species')
        else:
            sns.scatterplot(data=data, x='bill_length_mm', y='bill_depth_mm')
        plt.title(f'Bill Length vs Depth - {species}')
        plt.xlabel('Bill Length (mm)')
        plt.ylabel('Bill Depth (mm)')
    
    plt.tight_layout()
    
    # Return both plot and summary statistics
    summary = data.describe()
    
    return plt.gcf(), summary

# Get unique species for dropdown
species_options = ["All"] + list(penguins['species'].unique())

penguin_analyzer = gr.Interface(
    fn=analyze_penguins,
    inputs=[
        gr.Dropdown(species_options, value="All", label="Species"),
        gr.Radio(["Body Mass Distribution", "Bill Length vs Depth"], 
                value="Body Mass Distribution", label="Plot Type")
    ],
    outputs=[
        gr.Plot(label="Visualization"),
        gr.Dataframe(label="Summary Statistics")
    ],
    title="Penguin Data Explorer",
    description="Explore penguin data with interactive visualizations"
)

penguin_analyzer.launch()

### Example 2: Image Processing Tool

Let's create a simple image processing application:

In [ ]:
def process_image(image, brightness, contrast, rotate):
    if image is None:
        return None
    
    # Convert to PIL Image if needed
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)
    
    # Apply brightness adjustment
    from PIL import ImageEnhance
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(brightness)
    
    # Apply contrast adjustment
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(contrast)
    
    # Apply rotation
    if rotate != 0:
        image = image.rotate(rotate, expand=True)
    
    return image

image_processor = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(label="Upload Image"),
        gr.Slider(0.1, 2.0, value=1.0, step=0.1, label="Brightness"),
        gr.Slider(0.1, 2.0, value=1.0, step=0.1, label="Contrast"),
        gr.Slider(-180, 180, value=0, step=15, label="Rotation (degrees)")
    ],
    outputs=gr.Image(label="Processed Image"),
    title="Image Processor",
    description="Upload an image and adjust brightness, contrast, and rotation"
)

image_processor.launch()

## Advanced Features

### 1. Multiple Inputs and Outputs

Gradio can handle functions with multiple inputs and outputs:

In [ ]:
def data_summary_and_plot(csv_file, column_name, plot_type):
    if csv_file is None:
        return "Please upload a CSV file", None, None
    
    try:
        df = pd.read_csv(csv_file.name)
        
        if column_name not in df.columns:
            return f"Column '{column_name}' not found in the dataset", None, df.head()
        
        # Generate summary
        summary = f"""
        Dataset Summary:
        • Shape: {df.shape[0]} rows × {df.shape[1]} columns
        • Column '{column_name}' statistics:
          - Mean: {df[column_name].mean():.2f}
          - Median: {df[column_name].median():.2f}
          - Std Dev: {df[column_name].std():.2f}
          - Min: {df[column_name].min():.2f}
          - Max: {df[column_name].max():.2f}
        """
        
        # Generate plot
        plt.figure(figsize=(10, 6))
        if plot_type == "Histogram":
            plt.hist(df[column_name].dropna(), bins=20, alpha=0.7)
            plt.title(f'Distribution of {column_name}')
        elif plot_type == "Box Plot":
            plt.boxplot(df[column_name].dropna())
            plt.title(f'Box Plot of {column_name}')
        
        plt.xlabel(column_name)
        plt.tight_layout()
        
        return summary, plt.gcf(), df.head(10)
        
    except Exception as e:
        return f"Error processing file: {str(e)}", None, None

csv_analyzer = gr.Interface(
    fn=data_summary_and_plot,
    inputs=[
        gr.File(label="Upload CSV File", file_types=[".csv"]),
        gr.Textbox(label="Column Name", placeholder="Enter column name for analysis"),
        gr.Radio(["Histogram", "Box Plot"], value="Histogram", label="Plot Type")
    ],
    outputs=[
        gr.Textbox(label="Summary"),
        gr.Plot(label="Visualization"),
        gr.Dataframe(label="Data Preview")
    ],
    title="CSV Data Analyzer",
    description="Upload a CSV file and analyze a specific column"
)

csv_analyzer.launch()

## Try it on your own data!

Let's create a comprehensive dashboard that combines multiple features:

## Conclusion

Gradio provides a powerful and intuitive way to create web applications from Python functions. Key takeaways:


### Additional Resources:
- [Official Gradio Documentation](https://gradio.app/docs/)
- [Gradio Community Examples](https://huggingface.co/spaces?search=gradio)